In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage
from langchain.document_loaders import PyPDFLoader, NotionDirectoryLoader
from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from dotenv import load_dotenv
load_dotenv()

import getpass
import os


In [4]:

# PDF 파일 경로 설정
file_path = "/Users/munseunghyeon/chris_news/[시장동향분석][한국농수산식품유통공사][2017]농유공_2017 농식품 해외시장 맞춤조사_(인도_개사료)_(20170707102802).pdf"

# PDF 로더 초기화
loader = PyPDFLoader(file_path, extract_images=False)
pages = loader.load()

# 텍스트 분할기 초기화
text_splitter = CharacterTextSplitter(
    separator="\n", chunk_size=1000, chunk_overlap=150, length_function=len
)
docs = text_splitter.split_documents(pages)

# LLM 초기화
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# llm 모델을 가져와서 사용함
chat = ChatUpstage(upstage_api_key = os.getenv("UPSTAGE_API_KEY"))  


# 시스템 프롬프트 설정
contextualize_q_system_prompt = (
    "Given texts reformulate into a paragraph. "
    "formulate a standalone question which can be understood "
    "just reformulate it if needed and otherwise return it as is."
    "when you need more informations, don't ask just reformulate as much as you can"
)

# 사용자 프롬프트 설정
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        # ("chat_history", "Here would be the chat history if needed"),
        ("human", "{input}"),
    ]
)
parser = StrOutputParser()


# 체인 생성
chain = contextualize_q_prompt | chat | parser

# 체인 호출 및 결과 출력
result = chain.invoke({"language": "korean", "input": "hi"})
print(result)


Hello! How can I assist you today?


In [5]:
import time

target = ''
summary_result = []

for idx, text in enumerate(docs) :
    target += text.page_content
    if idx % 4  == 0 : 
        summary_result.append(chain.invoke(target))
        


In [20]:
summary_result

['2017년 농식품 해외시장 맞춤조사에서, 인도의 개사료 시장에 대한 정보를 제공하나요?',
 '인도에서 경쟁하는 개 사료 제품의 정량 분석, 정성 분석, 현지 방문 조사 자료는 어떻게 되나요?',
 '인도 개사료 시장의 2016년 기준 시장규모는 얼마인가요?',
 '2016년 기준 인도 내 HS CODE 2309.10.00 품목의 최대 수입국은 어디이며, 그 금액은 얼마인가요?',
 '인도 내 개사료 시장은 현재 어떤 상황인가요?',
 '인도 개사료 시장은 현재 규모가 작고 성장 속도가 느리지만, 반려견을 기르는 것에 대한 관심이 높아지고 있어 전망이 밝습니다. 인도에서는 매년 약 60만 마리의 반려견이 증가하고 있으며, 전체 인구 중 약 5%가 개를 기르고 있습니다. 이에 따라 애완동물 사료 및 관련 용품 시장의 성장이 예상됩니다. 특히, 최근에는 도시 부유층 사이에서 프리미엄 사료를 선호하는 소비자층이 증가하고 있습니다. 인도에서는 종교적 이유로 사료의 수입 절차가 까다로우므로, 수출업체는 항상 제품의 성분에 주의해야 합니다. 인도 정부는 사료의 원료가 되는 곡물의 공급을 위해 지속적인 재정 및 정책적 지원을 할 것으로 예상됩니다. 그러나, 인도에서는 아직 사료에 대한 법률이나 규칙이 명확하게 규정되어 있지 않습니다.',
 '제품명: Pedigree(성인) : 닭고기 & 채소(3kg)\n제조사: Mars International India Pvt. Ltd.\n유형: 현지산\n100g 당 가격 (원): 276.25\n중량 (g): 3000\n유통기한 (개월): 12\n포장형태: Bag\n원료: 닭고기, 쌀, 옥수수, 대두박, 식물성 기름, 비트펄프, 소금, 미네랄, 비타민\n원산지: 인도\n\n제품명: Pedigree(강아지) : 닭고기 & 우유(3kg)\n제조사: Mars International India Pvt. Ltd.\n유형: 현지산\n100g 당 가격 (원): 276.25\n중량 (g): 3000\n유통기한 (개월): 12\n포장형태: Bag\